This notebook takes the well data stored in a local database and renders out a binary file with the well location data in EarthTime format.

In [1]:
import os, array, csv, json, math, random
from datetime import datetime

import db_settings
import psycopg2

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

In [10]:
conn = psycopg2.connect(database=db_settings.DB, user=db_settings.USER, password=db_settings.PASSWD, host=db_settings.HOST)
with conn:
    with conn.cursor() as cur:
        query = 'SELECT date, ST_X(location::geometry) AS lon, ST_Y(location::geometry) AS lat FROM wells '
        query += "WHERE type IN ('OIL', 'GAS', 'DRY HOLE', 'DISPOSAL', 'OILANDGAS')"

        try:
            cur.execute(query)
        except psycopg2.Error as e:
            print (query)
            print (e.pgerror)
        wells = cur.fetchall()
conn.close()

In [11]:
data = []
base_date = datetime(1970, 1, 1)
for well in wells:
    date, lon, lat = well
    x, y = LonLatToPixelXY([lon,lat])
    if date:
        date = datetime.combine(date, datetime.min.time())
        epochtime = (date - base_date).total_seconds()
    else:
        epochtime = 0
    data += [x,y,epochtime]
array.array('f', data).tofile(open('data/data.bin', 'wb'))